---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **SEM-EX Clinical Dataset Preparation**

### Import Libraries

In [212]:

import numpy as np
import torch
import pandas as pd
import os
import sys
os.chdir(r'C:\Users\Ruffi\Desktop\Deep_clustering_SEM-EX\src\models\Clustering_Correlation_Analysis')


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## IMPORT DATA FROM DIRECTORIES: PROSPETTICO
### 1) Patient Table
Sono importati i dati associati alle features comuni del dataset CLARO-PROSPETTICO

In [213]:
# In questo punto cerchiamo la tabella di informazioni di CLARO PROSPETTICO - Patients Table
directory_clinical_features = r'C:\Users\Ruffi\Desktop\PHD\PROGETTI\ANNO1\SEM-EX\DATA\data\CLARO\prospettico\CLINICAL FEATURES\CLARO_prospettico_patients_table.xlsx'
# CARICHIAMO IL FOGLIO DI FEATURES COMUNI
feature_comuni_Patient_Table = pd.read_excel(directory_clinical_features)




### PULIZIA DEL DATASET E SELEZIONE DELLE FETURES PRIMARIE

* Construction of training Dataset.
* Pulizia del dataset da feature mancanti
* creazione feature Età paziente


In [214]:
# Eliminiamo dal dataset le righe che non hanno la data di diagnosi e lo stato
feature_comuni_filter = feature_comuni_Patient_Table.dropna(subset=['Data Diagnosi', 'Stato'])
# Primo dataset Prospettico con features
# Lista Attributi di interesse nel dataset 'ID paziente', 'Sesso', 'Stadio alla diagnosi', 'Data di nascita', 'Data Diagnosi', 'Stato'

Clinical_Dataset_Final = feature_comuni_filter[['ID paziente', 'Sesso', 'Stadio alla diagnosi','Data di nascita', 'Data Diagnosi', 'Stato']].rename(columns={"Stadio alla diagnosi": 'Stadio'})


print(np.unique(Clinical_Dataset_Final['Stadio'].to_frame()))
# Sostituiamo lo Stadio con i valori: 'ii b','recidiva linfonodale', 'recidiva'; con il valore -> 'other'
Map_stadio = {'altro': 'altro',
              'iii a': 'iii',
              'iii b': 'iii',
              'iii c': 'iii',
              'iv': 'iv',
              'iv a': 'iv',
              'iv b':'iv',
              'ii b':'altro',
              'recidiva linfonodale':'altro',
              'recidiva':'altro'}

Clinical_Dataset_Final['Stadio'] = Clinical_Dataset_Final['Stadio'].map(Map_stadio)
# Aggiungiamo la colonna con l'età dei pazienti:
Clinical_Dataset_Final['Età'] = np.abs(
    Clinical_Dataset_Final['Data di nascita'] -
    Clinical_Dataset_Final['Data Diagnosi']
).astype('timedelta64[Y]').astype(int)
# Togliamo Dal Dataset le DATE: 'Data di Nascita', 'Data Diagnosi'
Clinical_Dataset_Final.sort_values(by=['ID paziente'],inplace=True)
# Ottengo la tabella con tutte le features necessarie:
Clinical_Dataset_Final= Clinical_Dataset_Final.reset_index(drop=True)
display(Clinical_Dataset_Final)

['ii b' 'iii a' 'iii b' 'iii c' 'iv' 'iv a' 'iv b' 'recidiva'
 'recidiva linfonodale']


,ID paziente,Sesso,Stadio,Data di nascita,Data Diagnosi,Stato,Età
0,14474,m,ivb,1947-11-24,2020-04-27,awd,72
1,58391,m,iiib,1944-03-05,2019-04-20,awd,75
2,81949,f,ivb,1950-01-27,2020-08-08,awd,70
3,105098,m,iv,1939-06-14,2019-05-28,dod,79
4,80119756,m,iiia,1935-08-11,2019-05-17,awd,83
...,...,...,...,...,...,...,...
163,1100041461,f,ivb,1989-07-14,2022-02-03,awd,32
164,1100041464,f,iiia,1947-03-13,2021-02-02,awd,73
165,1100049421,f,ivb,1945-01-18,2021-03-18,awd,76
166,1100060151,m,iiib,1941-02-21,2021-04-07,awd,80


### Estraiamo gli ID-Pazienti in un vettore separato dalla tabella Features Comuni

In [215]:
# Estraggo gli Id pazienti dalla colonna della tabella:
ID_pazienti_FC = Clinical_Dataset_Final['ID paziente'].to_frame().astype(int)
# In questo punto quello che faccio è trasformare il dataframe in una lista di ID numerici
list_ID_pazienti_FC = np.sort(list(ID_pazienti_FC['ID paziente']))

### 2) Clinical DATA: III & IV Stadio e Antomia Patologica
Sono importati i dati associati alle features comuni del dataset CLARO-PROSPETTICO

In [216]:

# In questo punto cerchiamo la tabella di informazioni di CLARO PROSPETTICO - Clinical DB
directory_clinical_features = r'C:\Users\Ruffi\Desktop\PHD\PROGETTI\ANNO1\SEM-EX\DATA\data\CLARO\prospettico\CLINICAL FEATURES\CLARO_prospettico_clinical_db.xlsx'
# CARICHIAMO IL FOGLIO DI FEATURES COMUNI
clinical_Data = pd.read_excel(directory_clinical_features)
# cambiamo gli indici della matrice e mettiamo un doppio indice alle colonne
columns = clinical_Data.columns.to_list(
)
# Creazione di un nuovo indice multiplo per il Dataframe
index_new_unique  = [column for column in columns if 'Unnamed' not in column ]
string = index_new_unique[0]
array_new_index = list()
# Primo Indice
for column in columns:
    if column in index_new_unique:
        string = column
    array_new_index.append(string)
# Secondo indice
first_row = clinical_Data.loc[0,:].to_frame().reset_index()
second_row = clinical_Data.loc[1,:].to_frame().reset_index()
first_row.loc[0] = second_row.iloc[0][1]

# Indici insieme
arrays = [
    list(array_new_index),
    list(first_row.T.loc[0,:])
]
# Tupla scomposta ( taglio il vettore N * 2  in N tuple da 2 elementi )
tuples = list(zip(*arrays))
# Creazione del Dataframe modificato con colonne in MultiIndex di pandas
clinical_Data.columns = arrays
clinical_Data = clinical_Data.drop([0,1]).reset_index(drop=True)
# Select Clinical Data

### 1)  Estraiamo gli ID-Pazienti in un vettore separato dalla tabella Features cliniche

In [217]:

# CLinical Database Patients
ID_pazienti_CD = clinical_Data.feature_comuni['ID paziente'].astype(int)
# Lista di tutt gli ID pazienti nel dataset raw
list_ID_pazienti_CD = np.sort(list(ID_pazienti_CD))


## 2) Separazione del Clinical dataset di Claro prospettico rispetto i diversi fogli del foglio excel
Elimino Dal dataset i pazienti non presenti nelle features finali

In [218]:
# Separiamo i diversi Dataframe secondo le fasi di interesse
# 0) Features Cliniche di Base da Patient Table / ID pazienti selezionati rispetto filtro base su Data Diagnosi e stato / Clinical Data
# Scarto i pazienti non presenti in ID_pazienti_FC dal DB Clinico e li ordino secondo l'id Paziente:
clinical_Data[(     'feature_comuni',                        'ID paziente')] = clinical_Data[(     'feature_comuni',                        'ID paziente')].astype(int)
display(clinical_Data[(     'feature_comuni',                        'ID paziente')].sort_values())
Clinical_DB = clinical_Data.sort_values(by=[(     'feature_comuni',                        'ID paziente')])
Clinical_DB = Clinical_DB[[patient in list_ID_pazienti_FC for patient in list_ID_pazienti_CD]].reset_index(drop=True)
display(Clinical_DB)

125         14474
91          15803
85          28142
15          58391
152         81949
          ...    
201    1100041461
206    1100041464
202    1100049421
203    1100060151
208    1100069220
Name: (feature_comuni, ID paziente), Length: 209, dtype: int32

feature_comuni                                \
       ID paziente         CRA TAC utilizzabile?   
0            14474    96014639                si   
1            58391  CC01002254                si   
2            81949  CC02011152                si   
3           105098  CC04002982                si   
4         80119756  CC07006641                no   
..             ...         ...               ...   
163     1100041461         NaN               NaN   
164     1100041464         NaN               NaN   
165     1100049421         NaN               NaN   
166     1100060151         NaN               NaN   
167     1100069220         NaN               NaN   

                                                                             \
    Studio: retrospettivo/ prospettico Sesso      Data di nascita Peso (kg)   
0                          prospettico     m  1947-11-24 00:00:00       NaN   
1                          prospettico     m  1944-03-05 00:00:00        82   
2                          prospettico     f  1950-01-27 00:00:00       NaN   
3                          prospettico     m  1939-06-14 00:00:00       NaN   
4                          prospettico     m  1935-08-11 00:00:00       NaN   
..                                 ...   ...                  ...       ...   
163                        prospettico     f  1989-07-14 00:00:00       NaN   
164                        prospettico     f  1947-03-13 00:00:00        56   
165                        prospettico     f  1945-01-18 00:00:00       NaN   
166                        prospettico     m  1941-02-21 00:00:00        52   
167                        prospettico     m  1941-07-30 00:00:00       NaN   

                                                                      ...  \
    Altezza (cm)              Arruolato da... Provenienza Istologico  ...   
0            NaN                2 = oncologia                    NaN  ...   
1            163  1 = radioterapia oncologica                0 = cbm  ...   
2            NaN                2 = oncologia                    NaN  ...   
3            NaN                2 = oncologia                    NaN  ...   
4            NaN  1 = radioterapia oncologica                    NaN  ...   
..           ...                          ...                    ...  ...   
163          NaN                2 = oncologia                    NaN  ...   
164          162  1 = radioterapia oncologica                0 = cbm  ...   
165          NaN                2 = oncologia                    NaN  ...   
166          160  1 = radioterapia oncologica            1 = altrove  ...   
167          NaN  1 = radioterapia oncologica                    NaN  ...   

     anatomia_patologica                                                \
           Data prelievo          Sede prelievo   Modalità di prelievo   
0    2020-04-24 00:00:00             0= polmone         1= tac guidato   
1    2019-04-19 00:00:00             0= polmone         1= tac guidato   
2                    NaN  3= linfonodo toracico                2= tbna   
3    2019-05-27 00:00:00             0= polmone  3= altro (percutanea)   
4    2019-05-17 00:00:00             0= polmone         1= tac guidato   
..                   ...                    ...                    ...   
163                  NaN                    NaN                    NaN   
164                  NaN             0= polmone         1= tac guidato   
165                  NaN                    NaN                    NaN   
166                  NaN                    NaN                    NaN   
167                  NaN                    NaN                    NaN   

                                                                        \
                  Diagnosi    IHC Mutazione gene EGFR              ALK   
0        0= adenocarcinoma  1= si         8 = ex19del      1= negativo   
1    1= carcinoma squamoso  1= si    0 = non eseguito  0= non eseguito   
2        0= adenocarcinoma    NaN     

In [219]:
# 1) feature comuni
feature_comuni_CD = Clinical_DB.feature_comuni.copy(deep=True)
# 2) Terzo Stadio

III_stadio_Clinical_CD = Clinical_DB.iii_stadio.copy(deep=True)
# 3) Quarto Stadio

# Da Clinical DB
IV_stadio_Clinical_CD = Clinical_DB.iv_stadio.copy(deep=True)

# 4) Anatomia Patologica
An_Patologica = Clinical_DB.anatomia_patologica.copy(deep=True)





In [220]:
# Funzione per determinare il trattamento primario:
def boolean_RT_CT(RT, CT):
    RT = RT.strip().split('=')[-1]
    if RT == 'nan' and CT != 'nan':
        return CT
    elif RT != 'nan' and CT == 'nan':
        return RT
    elif RT != 'nan' and CT != 'nan':
        return RT + '/' + CT
    else:
        return 'nan'


# Estraiamo dal Dataset III e IV stadio il trattamento primario adottato al paziente
# Trattamento Tecnica Radio Terapia 3 stadio
Trattamento_Radio_Tecnica = III_stadio_Clinical_CD['Tecnica'].astype(str)
list_RT = Trattamento_Radio_Tecnica.to_list()
# Lista dei trattamenti cancro al polmone III stadio
print( np.unique(list_RT,return_counts=True))

# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Chirurgia III stadio/ Chemioterapia Concomitante
Trattamento_CT_III = III_stadio_Clinical_CD['CT conc SI/NO'].map({'si': 1, 'no': 0}).fillna(0)
list_trattamenti_III = Trattamento_CT_III.to_list()
print( np.unique(list_trattamenti_III,return_counts=True))


Trattamento_CHIRURGIA_III = III_stadio_Clinical_CD['Chirurgia dopo NAD'].map({'si': 1, 'no': 0}).fillna(0)
list_CH_III = Trattamento_CHIRURGIA_III.to_list()
print( np.unique(Trattamento_CHIRURGIA_III,return_counts=True))
# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Trattamento ChemioTerapico IV stadio
Trattamento_CT_IMM = IV_stadio_Clinical_CD['Prima linea di trattamento'].astype(str)
list_CT = Trattamento_CT_IMM.to_list()
print( np.unique(list_CT,return_counts=True))


# ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Cambiamo i nomi dei valori nei Trattamenti e nella Diagnosi di anatomia Patologica

# Creazione della colonna per i trattamenti primari
mapper_terapia= {' 3dcrt': 'radio',
          ' imrt':'radio',
          ' mista':'mista',
          ' vmat': 'radio',
          ' vmat/chemioterapia':'radio/chemio',
          ' vmat/immunoterapia':'radio/chemio',
          'chemio-immunoterapia':'chemio-immunoterapia',
          'chemioterapia':'chemioterapia',
          'immunoterapia':'immunoterapia',
          'nan':'altro',
          'terapia target':'terapia target'}
trattamenti_TumorePolmone = pd.Series([boolean_RT_CT(RT, CT) for RT, CT in zip(list_RT, list_CT)]).map(mapper_terapia)

# Mapper
print( np.unique(trattamenti_TumorePolmone,return_counts=True))
print(trattamenti_TumorePolmone)


# Creazione della colonna per la diagnosi Istologica del tumore
Colonna_Diagnosi_Ist = An_Patologica['Diagnosi'].astype(str)
keys = np.unique(Colonna_Diagnosi_Ist, return_counts=True)
# Dictionary Mapper for Dataframe ANATOMIA PATOLOGCIA
dictionary_mapper = {'0= adenocarcinoma': 'adenocarcinoma',
                     '1= carcinoma squamoso': 'carcinoma squamoso',
                     '2= carcinoma a piccole cellule': 'carcinoma',
                     '4= adenosquamoso': 'altro',
                     '5= nsclc, nas': 'nas',
                     '6= altro (carcinoma)': 'carcinoma',
                      '6= altro (neoplasia epiteliomorfa)': 'altro',
                     'nan': 'altro'}
Colonna_Diagnosi_Ist = Colonna_Diagnosi_Ist.map(dictionary_mapper)

# applichiamo il mapping anche ad altre colonne



# aggiungiamo la colonna alle feature finali


Clinical_Dataset_Final['Trattamento'] = trattamenti_TumorePolmone

Clinical_Dataset_Final['Diagnosi Istologica'] = Colonna_Diagnosi_Ist

Clinical_Dataset_Final['CT_concomitante'] = Trattamento_CT_III

Clinical_Dataset_Final['Chirurgia'] = Trattamento_CHIRURGIA_III

Clinical_Dataset_Final[['cT', 'cN', 'cM']] = feature_comuni_CD[['cT','cN','cM']]



(array(['0 = 3dcrt', '1 = vmat', '2 = imrt', '4 = mista', 'nan'],
      dtype='<U9'), array([ 1, 68,  1,  4, 94], dtype=int64))
(array([0., 1.]), array([100,  68], dtype=int64))
(array([0., 1.]), array([150,  18], dtype=int64))
(array(['chemio-immunoterapia', 'chemioterapia', 'immunoterapia', 'nan',
       'terapia target'], dtype='<U20'), array([32, 26, 18, 70, 22], dtype=int64))
(array(['chemio-immunoterapia', 'chemioterapia', 'immunoterapia', 'mista',
       'radio', 'radio/chemio', 'terapia target'], dtype=object), array([32, 23, 17,  4, 66,  4, 22], dtype=int64))
0            terapia target
1                     radio
2             chemioterapia
3             immunoterapia
4                     radio
               ...         
163          terapia target
164                   radio
165    chemio-immunoterapia
166                   radio
167                   radio
Length: 168, dtype: object


In [221]:
# Il dataframe finale con tutte le features di interesse:

Clinical_Dataset_prospettico = Clinical_Dataset_Final.copy(deep=True).rename(columns={'Diagnosi Istologica': 'Diagnosi'})
display(Clinical_Dataset_prospettico)

,ID paziente,Sesso,Stadio,Data di nascita,Data Diagnosi,Stato,Età,Trattamento,Diagnosi,CT_concomitante,Chirurgia,cT,cN,cM
0,14474,m,ivb,1947-11-24,2020-04-27,awd,72,terapia target,adenocarcinoma,0.0,0.0,NaN,NaN,NaN
1,58391,m,iiib,1944-03-05,2019-04-20,awd,75,radio,carcinoma squamoso,1.0,0.0,3,2,0
2,81949,f,ivb,1950-01-27,2020-08-08,awd,70,chemioterapia,adenocarcinoma,0.0,0.0,NaN,NaN,NaN
3,105098,m,iv,1939-06-14,2019-05-28,dod,79,immunoterapia,adenocarcinoma,0.0,0.0,NaN,NaN,NaN
4,80119756,m,iiia,1935-08-11,2019-05-17,awd,83,radio,adenocarcinoma,1.0,0.0,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1100041461,f,ivb,1989-07-14,2022-02-03,awd,32,terapia target,altro,0.0,0.0,NaN,NaN,NaN
164,1100041464,f,iiia,1947-03-13,2021-02-02,awd,73,radio,adenocarcinoma,1.0,0.0,2,2,0
165,1100049421,f,ivb,1945-01-18,2021-03-18,awd,76,chemio-immunoterapia,altro,0.0,0.0,NaN,NaN,NaN
166,1100060151,m,iiib,1941-02-21,2021-04-07,awd,80,radio,altro,1.0,0.0,4,2,0


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# IMPORT DATA FROM DIRECTORIES: Retrospettivo
## 1) Clinical Features
Sono importati i dati associati alle features comuni del dataset CLARO-PROSPETTICO

In [222]:
# In questo punto cerchiamo la tabella di informazioni di CLARO RETROSPETTIVO - Clinical DB
directory_clinical_features = r'C:\Users\Ruffi\Desktop\PHD\PROGETTI\ANNO1\SEM-EX\DATA\data\CLARO\retrospettivo\CLINICAL FEATURES\CLARO_retrospettivo_clinical_db.xlsx'
# CARICHIAMO IL FOGLIO DI FEATURES COMUNI
clinical_Data_retro = pd.read_excel(directory_clinical_features).sort_values(by=['ID paziente']).reset_index(drop=True).rename({'Tecnica': 'Trattamento'})
clinical_Data_retro

,ID paziente,Nr.,Iniziali paziente,nome,Luogo,cT,cN,Stadio alla diagnosi,CODE STADIO,CODE CT NAD,...,Data PFS,PFS,Cen PFS,Ultimo FUP,OS,Stato,Cens os,Note,Sesso,cM
0,9309,24,b.p.,Beconcini Paolina,longoni,2,2,iiia,0.0,1,...,2013-01-15,13.733333,0,2013-01-15,13.733333,doc,1,morta per insuff respiratoria (non abbiamo imm...,f,0
1,40158,15,d.c.a.,Di Censi Antonio,longoni,3,2,iiia,0.0,0,...,2011-09-20,31.033333,1,2012-01-17,35.000000,dod,1,"endoxan per recidiva, poi deceduto",m,0
2,51236,120,s.f.,Sabino Felicetta,campus,NaN,recidiva n2,iiia,0.0,0,...,2019-04-04,9.800000,0,2019-04-04,9.800000,awd,0,NaN,f,0
3,52384,28,f.g.,Forte Giuseppe,longoni,4,1,iiib,1.0,1,...,2012-11-27,23.066667,1,2014-10-24,46.266667,ned,0,prosegue controlli col dott. alberti,m,0
4,103515,100,c.r.,Cucchi Roberto,campus,3,2,iiib,1.0,0,...,2019-05-10,11.733333,0,2019-05-10,11.733333,awd,0,NaN,m,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,81774643,124,s.a.,Scardetta Antonio,campus,4,1,iiia,0.0,1,...,2019-03-07,6.266667,0,2019-03-07,6.266667,awd,0,NaN,m,0
126,100151470,73,c.m.,Carletti Mario,longoni,3,3,iiib,1.0,1,...,2012-09-11,6.100000,1,2012-11-08,8.033333,dod,1,progressione linfangite neoplastica,m,0
127,100189360,19,p.g.,Pierosara Giancarlo,longoni,4,2,iiib,1.0,1,...,2012-09-25,8.666667,0,2012-09-25,8.666667,doc,1,"morto per ascite cirrotico, scabbia",m,0
128,100237360,14,m.r.,Missimi Rosa,longoni,2,2,iiia,0.0,0,...,2012-06-06,13.533333,1,2019-06-06,98.733333,awd,0,ct in corso presso nostro dh,f,0


In [223]:


# Eliminiamo dal dataset le righe che non hanno la data di diagnosi e lo stato

feature_comuni_filter = clinical_Data_retro.dropna(subset=['Data Diagnosi', 'Stato'])
# Primo dataset Prospettico con features
# 1) 'ID paziente', 'Sesso', 'Stadio alla diagnosi', 'Data di nascita', 'Data Diagnosi', 'Stato'

Clinical_Dataset_retrospettivo = clinical_Data_retro[['ID paziente', 'Sesso', 'Stadio alla diagnosi', 'Data di nascita', 'Data Diagnosi', 'Stato','Tecnica', 'Diagnosi', 'cT', 'cN']].rename(columns={"Stadio alla diagnosi": 'Stadio'}).rename({'Diagnosi': 'Diagnosi Istologica'})

Clinical_Dataset_retrospettivo['Età'] =  clinical_Data_retro['Età'].astype(int)


print(np.unique(Clinical_Dataset_retrospettivo['Stato'].to_frame(), return_counts=True))
print(np.unique(Clinical_Dataset_retrospettivo['Diagnosi'].astype(str).to_frame(), return_counts=True))
# Mappiamo i tipi di tumore nei tipi presenti in retrospettivo
dictionary_mapper_Diag = {'adenocarcinoma': 'adenocarcinoma',
                     'carcinoma squamoso': 'carcinoma squamoso',
                     'carcinoma a grandi cellule': 'carcinoma',
                     'nos': 'nas',
                     'nan': 'altro'}
Clinical_Dataset_retrospettivo['Diagnosi']= Clinical_Dataset_retrospettivo['Diagnosi'].map(dictionary_mapper_Diag)
Clinical_Dataset_retrospettivo.rename(columns={'Tecnica': 'Trattamento'}, inplace=True)
# Mappiamo Tutte le terapie in Radio + NAD
Clinical_Dataset_retrospettivo['Trattamento'] = Clinical_Dataset_retrospettivo['Trattamento'].map({'3d':'radio', '3d crt':'radio', '3d+ra':'mista',  'ra':'radio'}).fillna('altro')

mapper_terapia= {' 3dcrt': 'radio',
          ' imrt':'radio',
          ' mista':'mista',
          ' vmat': 'radio',
          ' vmat/chemioterapia':'radio/chemio',
          ' vmat/immunoterapia':'radio/chemio',
          'chemio-immunoterapia':'chemio-immunoterapia',
          'chemioterapia':'chemioterapia',
          'immunoterapia':'immunoterapia',
          'nan':'altro',
          'terapia target':'terapia target'}


display(Clinical_Dataset_retrospettivo)

# Mappiamo le tecniche
print(np.unique(Clinical_Dataset_retrospettivo['Trattamento'].astype(str).to_frame(), return_counts=True))


# Estraiamo le features legate alla chirurgia e alla CT concomitante:

# Chirurgia III stadio/ Chemioterapia Concomitante
Trattamento_CT_retro = clinical_Data_retro['CODE CT NAD'].fillna(0).astype(float)
list_trattamenti_retro_CT = Trattamento_CT_retro.to_list()
print( np.unique(list_trattamenti_retro_CT,return_counts=True))


Trattamento_CHIRURGIA_retro = clinical_Data_retro['Chirurgia'].fillna(0).astype(float)
list_CH_retro = Trattamento_CHIRURGIA_retro.to_list()
print( np.unique(list_CH_retro,return_counts=True))

Clinical_Dataset_retrospettivo['cM'] = pd.DataFrame(np.zeros(list_CH_retro.__len__()))
Clinical_Dataset_retrospettivo['CT_concomitante'] = Trattamento_CT_retro
Clinical_Dataset_retrospettivo['Chirurgia'] = Trattamento_CHIRURGIA_retro


(array(['awd', 'doc', 'dod', 'ned'], dtype=object), array([40, 11, 51, 28], dtype=int64))
(array(['adenocarcinoma', 'carcinoma a grandi cellule',
       'carcinoma squamoso', 'nan', 'nos'], dtype=object), array([72,  3, 49,  3,  3], dtype=int64))


,ID paziente,Sesso,Stadio,Data di nascita,Data Diagnosi,Stato,Trattamento,Diagnosi,cT,cN,Età
0,9309,f,iiia,1925-07-28,2011-11-30,doc,radio,carcinoma squamoso,2,2,86
1,40158,m,iiia,1936-07-14,2009-03-03,dod,radio,carcinoma squamoso,3,2,72
2,51236,f,iiia,1960-06-03,2018-06-14,awd,radio,adenocarcinoma,NaN,recidiva n2,58
3,52384,m,iiib,1937-03-08,2011-01-05,ned,radio,carcinoma squamoso,4,1,73
4,103515,m,iiib,1946-10-11,2018-05-23,awd,radio,adenocarcinoma,3,2,71
...,...,...,...,...,...,...,...,...,...,...,...
125,81774643,m,iiia,1948-03-27,2018-08-31,awd,altro,carcinoma squamoso,4,1,70
126,100151470,m,iiib,1934-05-20,2012-03-12,dod,altro,adenocarcinoma,3,3,77
127,100189360,m,iiib,1948-05-20,2012-01-09,doc,radio,carcinoma squamoso,4,2,63
128,100237360,f,iiia,1940-12-27,2011-04-27,awd,radio,carcinoma squamoso,2,2,70


(array(['altro', 'mista', 'radio'], dtype=object), array([43,  8, 79], dtype=int64))
(array([0., 1.]), array([69, 61], dtype=int64))
(array([0., 1.]), array([100,  30], dtype=int64))


# Mergiamo i Due Dataset PRO-RETRO e poi selezioniamo gli ID pazienti che ci servono

In [224]:
# Mergiamo i due dataset secondo le features ottenute
Dataset_Clinical_RetroPro = pd.concat([Clinical_Dataset_retrospettivo, Clinical_Dataset_prospettico], ignore_index=True).sort_values(by=['ID paziente']).reset_index(drop=True)
# I dataset Clinici PRO e Retro sono stati concatenati insieme:
Dataset_Clinical_RetroPro
# Selezioniamo solo i pazienti che ci interessano a noi

list_Patient_final = Dataset_Clinical_RetroPro['ID paziente'].astype(int).to_numpy()

list_Patient_final

array([      9309,      14474,      40158,      51236,      52384,
            58391,      81949,     103515,     105098,   80057917,
         80060623,   80076045,   80093640,   80096063,   80099397,
         80117772,   80119756,   80125013,   80131109,   80131835,
         80132554,   80136256,   80143425,   80164869,   80187155,
         80191611,   80195995,   80210284,   80210790,   80211143,
         80214310,   80214678,   80251605,   80260907,   80409298,
         80416707,   80420560,   80431330,   80432920,   80435230,
         80461923,   80493676,   80504615,   80540094,   80586314,
         80650052,   80676115,   80683238,   80698764,   80707036,
         80753534,   80786334,   80842815,   80846016,   80859301,
         80859691,   80865913,   80867253,   80868958,   80881989,
         80891551,   80895174,   80896799,   80906854,   80919954,
         80927147,   80928094,   80933414,   80934874,   80941954,
         80946894,   80978654,   81003495,   81004714,   81011

# IMPORT DATA FROM DIRECTORIES: Clustering Data

In [225]:

# In questo punto cerchiamo la tabella di informazioni di CLARO Clusterizzato
directory_clustering_data = r'C:\Users\Ruffi\Desktop\Deep_clustering_SEM-EX\reports\data_clusters_labels_K_15_.xlsx'
# CARICHIAMO IL FOGLIO DI FEATURES COMUNI
cl_data = pd.read_excel(directory_clustering_data)

cl_data.drop_duplicates(subset=['patient ID'], inplace=True)
#
ID_pazienti_cl = cl_data['patient ID'].astype(int).to_numpy()
# Lista di tutt gli ID pazienti nel dataset raw
list_ID_pazienti_cl = np.sort(list(ID_pazienti_cl))


# Filter the Dataframe by the presence in the Final Features
Dataset_Clinical_RetroPro = Dataset_Clinical_RetroPro[[patient in list_ID_pazienti_cl for patient in list_Patient_final]].drop_duplicates(subset=['ID paziente']).sort_values(by=['ID paziente']).reset_index(drop=True).drop(columns=['Data Diagnosi', 'Data di nascita'])
display(Dataset_Clinical_RetroPro)

,ID paziente,Sesso,Stadio,Stato,Trattamento,Diagnosi,cT,cN,Età,cM,CT_concomitante,Chirurgia
0,9309,f,iiia,doc,radio,carcinoma squamoso,2,2,86,0.0,1.0,0.0
1,40158,m,iiia,dod,radio,carcinoma squamoso,3,2,72,0.0,0.0,0.0
2,51236,f,iiia,awd,radio,adenocarcinoma,NaN,recidiva n2,58,0.0,0.0,0.0
3,52384,m,iiib,ned,radio,carcinoma squamoso,4,1,73,0.0,1.0,0.0
4,58391,m,iiib,awd,radio,carcinoma squamoso,3,2,75,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
182,100151470,m,iiib,dod,altro,adenocarcinoma,3,3,77,0.0,1.0,0.0
183,100189360,m,iiib,doc,radio,carcinoma squamoso,4,2,63,0.0,1.0,0.0
184,100237360,f,iiia,awd,radio,carcinoma squamoso,2,2,70,0.0,0.0,0.0
185,1000000429,m,iiia,awd,radio,carcinoma,1,2,49,0,1.0,0.0


In [227]:


# In questo punto cerchiamo la tabella di informazioni di CLARO Clusterizzato
directory_labels_data = r'C:\Users\Ruffi\OneDrive - Università Campus Bio-Medico di Roma\DB_claro_\CLARO CLINICAL DATA\material\dataset\2023-02-01 00-00\labels.csv'

labels_CLARO = pd.read_csv(directory_labels_data).sort_values(by=['ID'])
labels_ = labels_CLARO['label'].map({'censored': 1, 'not censored': 0}).to_frame()[[patient in list_ID_pazienti_cl for patient in labels_CLARO['ID'].astype(int).to_numpy()]].reset_index(drop=True)
Dataset_Clinical_RetroPro['label OS'] = labels_
display(Dataset_Clinical_RetroPro)

# Salviamo la tabella nella cartella Data
Dataset_Clinical_RetroPro.to_excel("data/RETRO_PR_Dataset_Clinical.xlsx",)



,ID paziente,Sesso,Stadio,Stato,Trattamento,Diagnosi,cT,cN,Età,cM,CT_concomitante,Chirurgia,label OS
0,9309,f,iiia,doc,radio,carcinoma squamoso,2,2,86,0.0,1.0,0.0,0
1,40158,m,iiia,dod,radio,carcinoma squamoso,3,2,72,0.0,0.0,0.0,0
2,51236,f,iiia,awd,radio,adenocarcinoma,NaN,recidiva n2,58,0.0,0.0,0.0,1
3,52384,m,iiib,ned,radio,carcinoma squamoso,4,1,73,0.0,1.0,0.0,1
4,58391,m,iiib,awd,radio,carcinoma squamoso,3,2,75,0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,100151470,m,iiib,dod,altro,adenocarcinoma,3,3,77,0.0,1.0,0.0,0
183,100189360,m,iiib,doc,radio,carcinoma squamoso,4,2,63,0.0,1.0,0.0,0
184,100237360,f,iiia,awd,radio,carcinoma squamoso,2,2,70,0.0,0.0,0.0,1
185,1000000429,m,iiia,awd,radio,carcinoma,1,2,49,0,1.0,0.0,1
